# Assignment 3

Import packages

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib notebook

Write 3D cube coordinates

In [ ]:
# Write 3D cube coordinates
X= np.array([[0,1,1,0,0,1,1,0],
             [0,0,1,1,0,0,1,1],
             [0,0,0,0,1,1,1,1]])
print(X)

Transformation matrices in homogeneous coordinates

In [ ]:
# Scaling matrix
def scaling_matrix(s):
    # Scales are given in numpy array s
    # s=np.array([s_x,s_y,s_z])
    
    # Write the scaling matrix here
    S = np.identity(4)
    S[:3, :3] = np.diag(s)
    
    return S

# Translation matrix
def translation_matrix(t):
    # Translation vector is given in numpy array t
    # t=np.array([t_x,t_y,t_z])
    
    # Write the translation matrix here
    T = np.identity(4)
    T[:3, 3] = t
    
    return T

# Rotation matrix
def rotation_matrix(theta, v):
    # Axis for rotation (unit norm vector) is given 
    # is given in numpy array v
    # v=np.array([v_x,v_y,v_z])
    # Rotation angle is given in theta
    
    # Write the rotation matrix here
    st = np.sin(theta)
    ct = np.cos(theta)
    o_ct = 1-ct
    vx2= v[0]**2
    vy2= v[1]**2
    vz2= v[2]**2
    vxy = v[0]*v[1]
    vxz = v[0]*v[2]
    vyz= v[1]*v[2]
    # Rotation
    R = [[ct+vx2*o_ct, vxy*o_ct-v[2]*st, vxz*o_ct+v[1]*st],
         [vxy*o_ct+v[2]*st, ct+vy2*o_ct, vyz*o_ct-v[0]*st],
         [vxz*o_ct-v[1]*st, vyz*o_ct+v[0]*st, ct+vz2*o_ct]]
    R_v = np.identity(4)
    R_v[:3, :3] = R
    
    return R_v

# Projection function 
def im_proj(X,K_cw,f,s,o,pix):
    
    # Number of points
    n_points = np.shape(X)
    
    # Add fourth coordinate to have homogeneous coordinates
    X_h = np.vstack((X,np.ones((1,n_points[1]),dtype=np.float)))
    
    # Write K_wc matrix
    K_wc = np.linalg.inv(K_cw)
    
    # Write K_f times Pi_0 matrix
    K_f  = np.identity(3)
    K_f[0,0] = f
    K_f[1,1] = f
    K_f_Pi_0 = np.hstack((K_f,np.zeros((3,1), dtype=float)))
    
    # Write K_s matrix
    K_s = np.identity(3,dtype=float)
    K_s[:2,2] = o
    K_s[0,0]  = s[0]
    K_s[1,1]  = s[1]
    K_s[0,1]  = s[2]
    
    # Write the camera matrix
    M = np.dot(K_s,np.dot(K_f_Pi_0,K_wc))
    
    # Projection in homogeneous coordinates
    X_im_h = np.dot(M,X_h)
    X_im = X_im_h[:2,:]/X_im_h[2,:]
    
    # Deleting points outside image buffer
    # Negative values
    out_buffer_1 = (X_im<0)
    # Out of the upper bounds
    out_buffer_2 = ((np.tile(np.atleast_2d(pix).T,(1,n_points[1]))-X_im)<0)
    # Fusion of the two criteria
    out_buffer   =  np.sum(np.vstack((out_buffer_1,out_buffer_2)),axis=0)
    # Indexes
    ind = np.nonzero(out_buffer)
    # Delete
    X_im = np.delete(X_im,ind,axis=1)
    
    # Floor outputs to integers
    X_im = X_im.astype(int)
    
    # Output image buffer
    im_buffer = np.zeros((pix[1].astype(int)-1,pix[0].astype(int)-1))
    x,y = zip(X_im)
    im_buffer[y,x] = 255.0
    
    return im_buffer, X_im

Writing translation and rotation of the camera with 
respect to world frame

In [ ]:
R_cw = rotation_matrix(0.0*np.pi/180.0,np.array([0.0,1.0,0.0]))
t_cw = translation_matrix(np.array([0.0,0.0,6.0]))
K_cw = np.dot(t_cw,R_cw)

Write internal parameters of the camera

In [ ]:
f = 1
s = np.array([600.0,-600.0,0.0])
pix = np.array([640.0,480.0])
o = pix/2.0

Apply image projection to the points

In [ ]:
im_buffer, X_im = im_proj(X,K_cw,f,s,o,pix)
# print(X_im)
plt.imshow(im_buffer,cmap='Greys');

Flip matrix to have realistic view

In [ ]:
plt.imshow(np.flipud(np.fliplr(im_buffer)),cmap='Greys');